# Basic

In [59]:
import os
from datetime import datetime, timedelta, timezone

import requests
import pandas as pd
import pandas_gbq
from dateutil.relativedelta import relativedelta
from google.cloud import bigquery
from dotenv import load_dotenv

load_dotenv()
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("expand_frame_repr", False)

In [48]:
API_KEY = os.environ["API_KEY"]
API_URL = os.environ["API_URL"]
ACCOUNT_ID = os.environ["ACCOUNT_ID"]
INSTRUMENT = "USD_JPY"
GRANULARITY = "D"
PRICE = "M"  # “M” (midpoint candles) “B” (bid candles) and “A” (ask candles)
ALIGNMENT_TIMEZONE = "America/New_York"
TABLE = "oanda.usdjpy_ny_day_mid_candles"

# Get candles

In [73]:
def make_headers():
    return {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json"
    }

def get_candles(from_str, to_str):
    headers = make_headers()
    payload = {
        "price": PRICE,
        "granularity": GRANULARITY,
        "from": from_str,
        "to": to_str,
        "alignmentTimezone": ALIGNMENT_TIMEZONE
    }
    r = requests.get(
        url=f"{API_URL}/v3/instruments/{INSTRUMENT}/candles",
        headers=headers,
        params=payload
    )
    
    # print("get_candles()")
    # print(f"Status code: {r.status_code}")
    # print(r.text)
    # pprint.pprint(r.json()["candles"])

    candles = r.json()["candles"]
    return candles


def candles_to_df(candles):
    data = []
    for candle in candles:
        d = dict()
        d["date"] = candle["time"][:10]
        d["open"] = candle["mid"]["o"]
        d["high"] = candle["mid"]["h"]
        d["low"] = candle["mid"]["l"]
        d["close"] = candle["mid"]["c"]
        d["volume"] = candle["volume"]
        data.append(d)

    df = pd.DataFrame(data)

    for c in ["open", "high", "low", "close", "volume"]:
        df[c] = pd.to_numeric(df[c])
    df["write_timestamp"] = datetime.now(timezone.utc)
    
    return df

In [29]:
res = get_candles(from_str="2024-01-01", to_str="2024-01-02")
print(type(res))
print(len(res))
for r in res:
    print(r)

<class 'list'>
1
{'complete': True, 'volume': 143363, 'time': '2024-01-01T22:00:00.000000000Z', 'mid': {'o': '140.858', 'h': '142.216', 'l': '140.811', 'c': '142.006'}}


In [58]:
res = get_candles(from_str="2002-01-01", to_str="2002-12-31")
print(type(res))
print(len(res))
for r in res[:5]:
    print(r)
for r in res[-5:]:
    print(r)

<class 'list'>
204
{'complete': True, 'volume': 1, 'time': '2002-05-06T21:00:00.000000000Z', 'mid': {'o': '127.920', 'h': '127.920', 'l': '127.920', 'c': '127.920'}}
{'complete': True, 'volume': 1, 'time': '2002-05-07T21:00:00.000000000Z', 'mid': {'o': '128.920', 'h': '128.920', 'l': '128.920', 'c': '128.920'}}
{'complete': True, 'volume': 1, 'time': '2002-05-08T21:00:00.000000000Z', 'mid': {'o': '128.380', 'h': '128.380', 'l': '128.380', 'c': '128.380'}}
{'complete': True, 'volume': 1, 'time': '2002-05-09T21:00:00.000000000Z', 'mid': {'o': '127.630', 'h': '127.630', 'l': '127.630', 'c': '127.630'}}
{'complete': True, 'volume': 1, 'time': '2002-05-10T21:00:00.000000000Z', 'mid': {'o': '127.620', 'h': '127.620', 'l': '127.620', 'c': '127.620'}}
{'complete': True, 'volume': 1, 'time': '2002-12-24T22:00:00.000000000Z', 'mid': {'o': '120.320', 'h': '120.320', 'l': '120.320', 'c': '120.320'}}
{'complete': True, 'volume': 1, 'time': '2002-12-25T22:00:00.000000000Z', 'mid': {'o': '119.970', '

# Upload candles to BigQuery

In [60]:
# d = datetime(2020, 1, 1)

# while d >= datetime(2002, 1, 1):
#     from_str = d.strftime("%Y-%m-%d")
#     to_str = (d + relativedelta(years=1) - timedelta(days=1)).strftime("%Y-%m-%d")
#     print(f"from: {from_str}, to: {to_str}")

#     res = get_candles(from_str=from_str, to_str=to_str)
#     df = candles_to_df(res)
#     print("Uploading...")
#     pandas_gbq.to_gbq(df, destination_table=TABLE, if_exists="append")
#     print("Finished")
    
#     d -= relativedelta(years=1)

# Check candles in BigQuery

In [62]:
query = """
SELECT *
FROM `oanda.usdjpy_ny_day_mid_candles`
ORDER BY date
"""

df = bigquery.Client().query(query).to_dataframe()

C:\Users\yk279\PycharmProjects\oanda\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [63]:
df.shape

(6707, 7)

In [64]:
df.head()

,date,open,high,low,close,volume,write_timestamp
0,2002-05-06,127.92,127.92,127.92,127.92,1,2025-07-05 23:41:41.882727+00:00
1,2002-05-07,128.92,128.92,128.92,128.92,1,2025-07-05 23:41:41.882727+00:00
2,2002-05-08,128.38,128.38,128.38,128.38,1,2025-07-05 23:41:41.882727+00:00
3,2002-05-09,127.63,127.63,127.63,127.63,1,2025-07-05 23:41:41.882727+00:00
4,2002-05-10,127.62,127.62,127.62,127.62,1,2025-07-05 23:41:41.882727+00:00


In [65]:
df.tail()

,date,open,high,low,close,volume,write_timestamp
6702,2024-12-23,157.129,157.394,156.890,157.210,131604,2025-07-05 23:29:46.536036+00:00
6703,2024-12-25,157.257,158.087,157.077,158.023,141678,2025-07-05 23:29:46.536036+00:00
6704,2024-12-26,157.910,157.953,157.352,157.843,182121,2025-07-05 23:29:46.536036+00:00
6705,2024-12-29,157.747,158.074,156.668,156.860,204705,2025-07-05 23:29:46.536036+00:00
6706,2024-12-30,156.887,157.550,156.020,157.203,228120,2025-07-05 23:29:46.536036+00:00


# Upload a candle to BigQuery

In [79]:
# from_dt = datetime.now(timezone.utc) - timedelta(days=1)
from_dt = datetime(2025, 7, 4)
from_str = from_dt.strftime("%Y-%m-%d")

# to_dt = datetime.now(timezone.utc)
to_dt = datetime(2025, 7, 5)
to_str = to_dt.strftime("%Y-%m-%d")

print(f"from: {from_str}, to: {to_str}")

res = get_candles(from_str=from_str, to_str=to_str)
df = candles_to_df(res)

if df.iloc[0]["date"] == from_str:
    print("Uploading...")
    pandas_gbq.to_gbq(df, destination_table=TABLE, if_exists="append")
    print("Finished")
else:
    print("Date doesn't match, so don't upload")

df

from: 2025-07-04, to: 2025-07-05
Date doesn't match, so don't upload


,date,open,high,low,close,volume,write_timestamp
0,2025-07-03,144.928,144.974,144.183,144.491,202107,2025-07-06 00:14:34.164472+00:00
